# Data Curation Part 1 - Cleaning and Filtering

This part filters and cleans the Swiss Dwellings (SD) dataset using the following steps (same as [paper](https://arxiv.org/abs/2407.10121); copy-pasted text):

[$\bullet$] **Feature removal.**
All non-floorplan geometries are removed (\eg "bathtub", "stairs"; see the full list in the suppl. mat.).

[$\bullet$] **Residential-only filtering.**
We remove floor plans that include non-residential-like geometric details (\eg areas categorized as "waiting room", "dedicated medical room"; see the full list in the suppl. mat.).
This led to the removal of 2,305 (16.6\%) floor plans.

[$\bullet$] **Near-duplicate removal.**
Many floor plans that come from the same building stem from the same plan ID~\cite{standfest_swiss_2022} (see suppl. mat. for details on ID nesting in SD).
Floor plans with the same plan ID are based on the same layout, indicating that the spatial arrangements are nearly identical.
Hence, we sample only one-floor plan per plan ID to drastically reduce the amount of near-duplicates.
Specifically, we sample the floor plan with the lowest elevation.
This led to the removal of 4,395 (31.6\%) floor plans.

[$\bullet$] **Medium- to large-scale filtering.**
Floor plans are removed that contain fewer than 15 areas. In addition, every floor plan should have at least two "Zone 2"-categorized areas.
This led to the removal of 1,541 (11.1\%) floor plans.

Note that the computation needed for the filtering and cleaning is done in mere seconds.

## Loading and acquiring information

You need to download the Swiss Dwellings (SD) `geometries.csv` file here: [SD 3.0.0 (latest version)](https://zenodo.org/records/7788422).

We extensively use `Pandas` and `GeoPandas` for filtering and cleaning of the dataset. In the following code block, we load the complete table as a `GeoDataFrame`:

In [9]:
import os
import pandas as pd
from geopandas import GeoDataFrame as gdf

# Path to dataset
path = r'C:\Users\caspervanengel\OneDrive\Documents\PHD\1_data\swiss-dwellings\3.0.0'

# Load SD `geometries.csv` as geodataframe
DF = gdf(pd.read_csv(os.path.join(path, 'geometries.csv')))
DF = DF.rename(columns={'geometry': 'geom'})
df = DF.copy()

For an in-depth understanding on the columns, please refer to the [documentation of SD](https://zenodo.org/records/7788422).

We print some meta-level information of SD:

In [10]:
# Entity subtypes and types lists
entity_subtypes = set(df.entity_subtype)
entity_types = set(df.entity_type)

# Print information; be aware that subtypes are capitalized in the real dataset
print(f'Subtypes:\n\t{[cat.lower() for cat in entity_subtypes]}\n')
print(f'Types:\n\t{[cat.lower() for cat in entity_types]}\n')

# Print information on dimensions
# e.g., Number of distinct apartments, sites, buildings, etc.
print(f'Number of apartments:\t{len(df.apartment_id.unique())}')
print(f'Number of sites:\t\t{len(df.site_id.unique())}')
print(f'Number of buildings:\t{len(df.building_id.unique())}')
print(f'Number of floors:\t\t{len(df.floor_id.unique())}')
print(f'Number of plans:\t\t{len(df.plan_id.unique())}')
print(f'Number of units:\t\t{len(df.unit_id.unique())}')
print(f'Number of areas:\t\t{len(df.area_id.unique())}')
print(f'Number of geometries:\t{df.geom.count()}')

Subtypes:
	['medical_room', 'entrance_door', 'loggia', 'shelter', 'ramp', 'elevator_facilities', 'wash_and_dry_room', 'salesroom', 'bike_storage', 'air', 'room', 'balcony', 'office_space', 'railing', 'community_room', 'washing_machine', 'patio', 'vehicle_traffic_area', 'open_plan_office', 'bathroom', 'corridors_and_halls', 'radation_therapy', 'kitchen_dining', 'pram_and_bike_storage_room', 'transport_shaft', 'cloakroom', 'shower', 'common_kitchen', 'carpark', 'cold_storage', 'basement_compartment', 'built_in_furniture', 'lightwell', 'water_supply', 'foyer', 'arcade', 'bedroom', 'dedicated_medical_room', 'break_room', 'not_defined', 'meeting_room', 'archive', 'heating', 'living_dining', 'gas', 'reception_room', 'corridor', 'dining', 'staircase', 'teaching_room', 'electrical_supply', 'outdoor_void', 'counter_room', 'lobby', 'office', 'sink', 'sanitary_rooms', 'bathtub', 'workshop', 'sports_rooms', 'void', 'kitchen', 'showroom', 'wintergarten', 'canteen', 'door', 'operations_facilities', 

We are most interested in the nesting / tree of (sub)types: I.e., which subtypes are children of which types (parents).

In [11]:
for entity_type in entity_types:
    print(f'Type: {entity_type}\n'
          f'-- Subtypes:\n\t{[cat.lower() for cat in set(df[df["entity_type"]==entity_type].entity_subtype)]}\n')

Type: feature
-- Subtypes:
	['sink', 'washing_machine', 'built_in_furniture', 'kitchen', 'shower', 'bathtub', 'stairs', 'ramp', 'toilet', 'elevator']

Type: opening
-- Subtypes:
	['door', 'entrance_door', 'window']

Type: area
-- Subtypes:
	['medical_room', 'loggia', 'shelter', 'elevator_facilities', 'wash_and_dry_room', 'oil_tank', 'salesroom', 'bike_storage', 'air', 'room', 'balcony', 'office_space', 'community_room', 'patio', 'vehicle_traffic_area', 'open_plan_office', 'bathroom', 'corridors_and_halls', 'radation_therapy', 'kitchen_dining', 'pram_and_bike_storage_room', 'transport_shaft', 'cloakroom', 'common_kitchen', 'carpark', 'cold_storage', 'basement_compartment', 'lightwell', 'water_supply', 'foyer', 'arcade', 'bedroom', 'dedicated_medical_room', 'break_room', 'not_defined', 'meeting_room', 'archive', 'heating', 'living_dining', 'gas', 'reception_room', 'corridor', 'dining', 'staircase', 'teaching_room', 'electrical_supply', 'outdoor_void', 'counter_room', 'lobby', 'office', '

## Feature removal

This step is one of the most delicate steps of the process. It comes from the fact that there are two equivalently named *subtypes* that have a different *type* category. For the 'feature' and 'area' types, subtypes can be both 'KITCHEN' and 'ELEVATOR'. We first remove these two categories from the 'feature' type.

In [12]:
df = df[~(df['entity_type'] == 'feature')]
df = df[~(df['entity_subtype'] == 'RAILING')]

## Residential-only filtering

We remove floors that have at least one of the following subtypes:

```python
{'OFFICE', 'WASH_AND_DRY_ROOM', 'VEHICLE_TRAFFIC_AREA', 'BASEMENT', 'WAITING_ROOM',
'MEDICAL_ROOM', 'CANTEEN', 'SALESROOM', 'COLD_STORAGE', 'WORKSHOP', 'OUTDOOR_VOID',
'TECHNICAL_AREA', 'DEDICATED_MEDICAL_ROOM', 'WAREHOUSE', 'GARDEN', 'PRAM_AND_BIKE_STORAGE_ROOM',
'OFFICE_SPACE', 'SHOWROOM', 'AIR', 'PRAM', 'SANITARY_ROOMS', 'LOBBY', 'WATER_SUPPLY', 'HEATING',
 'LOGGIA', 'OIL_TANK', 'GAS', 'PATIO', 'FOYER', 'OPEN_PLAN_OFFICE', 'SHELTER', 'SPORTS_ROOMS',
 'HOUSE_TECHNICS_FACILITIES', 'TEACHING_ROOM', 'OFFICE_TECH_ROOM', 'WINTERGARTEN', 'CLOAKROOM',
 'RECEPTION_ROOM', 'CARPARK', 'STUDIO', 'NOT_DEFINED', 'COMMUNITY_ROOM',
 'PHYSIO_AND_REHABILITATION', 'FACTORY_ROOM', 'ARCHIVE', 'OPERATIONS_FACILITIES', 'ARCADE',
 'LOGISTICS', 'TRANSPORT_SHAFT', 'GARAGE', 'COUNTER_ROOM', 'BREAK_ROOM', 'ELECTRICAL_SUPPLY',
 'ELEVATOR_FACILITIES', 'MEETING_ROOM', 'COMMON_KITCHEN', 'BASEMENT_COMPARTMENT', 'LIGHTWELL',
 'BIKE_STORAGE', 'RADATION_THERAPY'}
```

In [13]:
from constants import ZONING_MAPPING

# Entity subtypes for which the floors should be completely removed
subtypes_to_remove = set(df.entity_subtype) - set(list(ZONING_MAPPING.keys()))

# Find floor ids of non- or mixed-residential
mixed_floor_ids = set(df[df['entity_subtype'].isin(set(subtypes_to_remove))].floor_id)
res_floor_ids = set(df.floor_id) - mixed_floor_ids  # The complement of the other

# Filter on residential-only floor ids
df = df[df['floor_id'].isin(res_floor_ids)].reset_index(drop=True)

# If you run it again; there won't, obviously, be any floor plans to-be-removed !
print(f'Number of deleted floors:\t{len(mixed_floor_ids)}')
print(f'Number of remaining floors:\t{len(df.floor_id.unique())}')

Number of deleted floors:	2305
Number of remaining floors:	11600


## Near-duplicate removal

We remove what we would like to call *near* duplicates.
It is important to not have too similar instances in the machine learning dataset because two (or more) of the same signals and annotations will never help (and will only hinder) training and might even cause imbalanced model behavior (usually per epoch the whole dataset is seen once; hence if there are duplicates the model will better learn on those instances) and unfair evaluation.

In [15]:
one_per_plan_floor_ids = df.sort_values('elevation').groupby('plan_id').head(1)['floor_id'].values
df = df[df['floor_id'].isin(one_per_plan_floor_ids)]

# after plan id selection
print(f'Number of remaining floors: {df.floor_id.nunique()}')

Number of remaining floors: 7205


## Medium- to large-scale filtering

Floor plans are removed that contain fewer than 15 areas. In addition, every floor plan should have at least two "Zone 2"-categorized areas.
This led to the removal of 1,541 (11.1\%) floor plans.

We first add two extra columns to the dataframe: One indicating the *zoning type* of the room; the other the *room type*.
See `constants.py` for the zoning and room type categories and mappings between them and the (original) subtype categories.

In [16]:
from constants import ROOM_MAPPING

df['zoning'] = df['entity_subtype'].map(ZONING_MAPPING)
df['roomtype'] = df['entity_subtype'].map(ROOM_MAPPING)

Now, we find all the floor identities (IDs) that consist of a minimum of 15 rooms and have at least 2 rooms of type `Zone 2`:

In [17]:
from tqdm.auto import tqdm

floor_ids = df.floor_id.unique()

min_nr_areas = 15
min_nr_zone2 = 2

# Initialize empty list
floor_ids_at_scale = []

# Takes a couple of seconds
for floor_id in tqdm(floor_ids):

    df_floor = df[df.floor_id == floor_id].reset_index(drop=True)
    df_floor = df_floor[df_floor.zoning.isin(['Zone1', 'Zone2', 'Zone3', 'Zone4'])]
    df_zone2 = df_floor[df_floor.zoning.isin(['Zone2'])]

    # Compute number of rooms and those with type Zone 2
    nr_areas = len(df_floor.zoning.values)
    nr_zone2 = len(df_zone2.zoning.values)

    # Filter on minimum number of areas and minimum number of Zone 2
    if nr_areas >= min_nr_areas and nr_zone2 >= min_nr_zone2:
        floor_ids_at_scale.append(floor_id)
    else:
        continue

  0%|          | 0/7205 [00:00<?, ?it/s]

And filter the dataframe based on these IDs:

In [18]:
# remove small complexes
df = df[df['floor_id'].isin(floor_ids_at_scale)].reset_index(drop=True)

# after removal of selection
print(f'Final number of floors: {df.floor_id.nunique()}')

# floors: 5664


## Saving the dataframe

You can now save the cleaned and filtered dataframe anywhere that you like.
In our GitHub, you can find it back under the folder `data`, saved as `f"MSD 5.664k (V1).csv"`

In [19]:
df.to_csv(f'data/MSD {len(set(df.floor_id))/1000:.3f}k (V1).csv')